In [1]:
import math
import tensorflow as tf
import numpy as np
import matplotlib
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)
xy_with_noise = np.genfromtxt('/Users/yeseo/Desktop/2015eliminated_1.txt',delimiter = ',',dtype = None)

#data_preprocessing
xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
b= MinMaxScaler(b)
xy = np.hstack((a,b))

xy_with_noise = xy_with_noise[:,:27]
a_with_noise = xy_with_noise[:,:2]
b_with_noise = xy_with_noise[:,2:]
b_with_noise = MinMaxScaler(b_with_noise)
xy_with_noise = np.hstack((a_with_noise,b_with_noise))


#parameters
seq_length =6
data_dim =27
hidden_dim = 25
output_dim = 25
learning_rate = 0.1
iterations = 1500

train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

#divide data set to train,validation and test set
train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

train_set_with_noise = xy_with_noise[:train_size]
validation_set_with_noise = xy_with_noise[train_size:train_size+validation_size]
test_set_with_noise = xy_with_noise[train_size+validation_size:]

# build data set for rnn
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

trainX_with_noise, trainY_with_noise = build_dataset(train_set_with_noise,seq_length)
validationX_with_noise, validationY_with_noise = build_dataset(validation_set_with_noise,seq_length)
testX_with_noise,testY_with_noise = build_dataset(test_set_with_noise, seq_length)


X1 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y1 = tf.placeholder(tf.float32,[None,25])

X2 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y2 = tf.placeholder(tf.float32,[None,25])

#LSTM CELL만들기

with tf.variable_scope("rnn1"):
    cell1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs1,_states1 = tf.nn.dynamic_rnn(cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[:,-1], output_dim,activation_fn = None)

    loss1 =tf.reduce_sum(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)

with tf.variable_scope("rnn2"):
    cell2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs2,_states2 = tf.nn.dynamic_rnn(cell2, X2, dtype = tf.float32)
    Y_pred_with_noise = tf.contrib.layers.fully_connected(outputs2[:,-1], output_dim,activation_fn = None)

    loss2 =tf.reduce_mean(tf.square(Y_pred_with_noise-Y2))
    train2 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss2)


#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
x2 = x1+0.3
x3 = x2+0.3
loss_for_graph = np.zeros(iterations)
x4 = np.array(range(0,iterations))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY})
        print("[step: {}] loss: {}".format(i,step_loss1))
        loss_for_graph[i] = step_loss1
        _, step_loss2 = sess.run([train2,loss2],feed_dict={X2:trainX_with_noise, Y2:trainY_with_noise})
        print("[step: {}] loss: {}".format(i,step_loss2))
        
    test_predict = sess.run(Y_pred, feed_dict = {X1:validationX})
    test_predict_with_noise = sess.run(Y_pred_with_noise, feed_dict = {X2:validationX_with_noise})

    
    rmse_val = sess.run(rmse, feed_dict={targets: validationY,predictions: test_predict})
    print("RMSE: {}".format(rmse_val))
   # print("pred: {}".format(test_predict[-1,:]))
    #print("real: {}".format(testY[-1,:]))
    #print("noise: {}".format(eliminate_noise_pred[-1,:]))
    
#    plt.bar(x1,test_predict[-1,:],label = 'predict',color ='b',width = 0.1)
  #  plt.bar(x2,testY[-1,:],label = 'real',color ='g',width = 0.1)
    #plt.bar(x3,eliminate_noise_pred[-1,:],label = 'noise',color ='g',width = 0.1)
    plt.plot(x4,loss_for_graph)
    #plt.legend()
    plt.show()



Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
[step: 0] loss: 19893.150390625
[step: 0] loss: 0.3186686038970947
[step: 1] loss: 95134.9375
[step: 1] loss: 0.18458418548107147
[step: 2] loss: 17048.6171875
[step: 2] loss: 0.07859673351049423
[step: 3] loss: 11760.109375
[step: 3] loss: 0.07895123213529587
[step: 4] loss: 11369.0849609375
[step: 4] loss: 0.046017102897167206
[step: 5] loss: 10146.640625
[step: 5] loss: 0.035002149641513824
[step: 6] loss: 9582.294921875
[step: 6] loss: 0.03080286830663681
[step: 7] loss: 7760.5107421875
[step: 7] loss: 0.02879856899380684
[step: 8] loss: 5162.228515625
[step: 8] loss: 0.028716888278722763
[step: 9] loss: 3604.385498046875
[step: 9] loss: 0.027990786358714104
[step: 10] loss: 3516.51220703125
[step: 10] loss: 0.026484794914722443
[step: 11] loss: 3786.322

[step: 109] loss: 251.46743774414062
[step: 109] loss: 0.015909329056739807
[step: 110] loss: 249.52658081054688
[step: 110] loss: 0.01590881310403347
[step: 111] loss: 247.75567626953125
[step: 111] loss: 0.015908051282167435
[step: 112] loss: 246.08767700195312
[step: 112] loss: 0.01590735651552677
[step: 113] loss: 244.48402404785156
[step: 113] loss: 0.01590673252940178
[step: 114] loss: 242.86795043945312
[step: 114] loss: 0.01590598002076149
[step: 115] loss: 241.33001708984375
[step: 115] loss: 0.015905452892184258
[step: 116] loss: 239.8590087890625
[step: 116] loss: 0.015904711559414864
[step: 117] loss: 238.48040771484375
[step: 117] loss: 0.01590409129858017
[step: 118] loss: 237.13572692871094
[step: 118] loss: 0.015903349965810776
[step: 119] loss: 235.84036254882812
[step: 119] loss: 0.015902595594525337
[step: 120] loss: 234.56808471679688
[step: 120] loss: 0.01590186357498169
[step: 121] loss: 233.34722900390625
[step: 121] loss: 0.015901023522019386
[step: 122] loss: 2

[step: 219] loss: 0.014916723594069481
[step: 220] loss: 425.25567626953125
[step: 220] loss: 0.01472877524793148
[step: 221] loss: 254.10108947753906
[step: 221] loss: 0.01468508131802082
[step: 222] loss: 186.93753051757812
[step: 222] loss: 0.014666436240077019
[step: 223] loss: 293.3839416503906
[step: 223] loss: 0.01421319879591465
[step: 224] loss: 297.58648681640625
[step: 224] loss: 0.014248916879296303
[step: 225] loss: 204.66769409179688
[step: 225] loss: 0.014653025195002556
[step: 226] loss: 219.4003448486328
[step: 226] loss: 0.013931499794125557
[step: 227] loss: 281.90728759765625
[step: 227] loss: 0.014665095135569572
[step: 228] loss: 219.70106506347656
[step: 228] loss: 0.017267493531107903
[step: 229] loss: 212.09161376953125
[step: 229] loss: 0.014887530356645584
[step: 230] loss: 246.6709747314453
[step: 230] loss: 0.028513319790363312
[step: 231] loss: 197.5556640625
[step: 231] loss: 0.02986745722591877
[step: 232] loss: 202.618896484375
[step: 232] loss: 0.03581

[step: 329] loss: 237.35440063476562
[step: 329] loss: 0.02154260128736496
[step: 330] loss: 177.5238800048828
[step: 330] loss: 0.02154085971415043
[step: 331] loss: 243.19723510742188
[step: 331] loss: 0.021539131179451942
[step: 332] loss: 190.27392578125
[step: 332] loss: 0.021538155153393745
[step: 333] loss: 210.43580627441406
[step: 333] loss: 0.021537845954298973
[step: 334] loss: 200.07501220703125
[step: 334] loss: 0.021537693217396736
[step: 335] loss: 177.21853637695312
[step: 335] loss: 0.021537363529205322
[step: 336] loss: 205.46478271484375
[step: 336] loss: 0.02153681218624115
[step: 337] loss: 170.89732360839844
[step: 337] loss: 0.021536286920309067
[step: 338] loss: 202.2667236328125
[step: 338] loss: 0.02153589390218258
[step: 339] loss: 170.05059814453125
[step: 339] loss: 0.021535595878958702
[step: 340] loss: 185.92852783203125
[step: 340] loss: 0.02153521217405796
[step: 341] loss: 175.77728271484375
[step: 341] loss: 0.02153473161160946
[step: 342] loss: 173.5

[step: 440] loss: 0.021528054028749466
[step: 441] loss: 148.93359375
[step: 441] loss: 0.02152802236378193
[step: 442] loss: 148.87013244628906
[step: 442] loss: 0.021528001874685287
[step: 443] loss: 148.83462524414062
[step: 443] loss: 0.021527981385588646
[step: 444] loss: 148.8587646484375
[step: 444] loss: 0.021527955308556557
[step: 445] loss: 149.0079345703125
[step: 445] loss: 0.021527914330363274
[step: 446] loss: 149.45394897460938
[step: 446] loss: 0.021527905017137527
[step: 447] loss: 150.55596923828125
[step: 447] loss: 0.021527869626879692
[step: 448] loss: 153.36813354492188
[step: 448] loss: 0.021527830511331558
[step: 449] loss: 159.97315979003906
[step: 449] loss: 0.021527796983718872
[step: 450] loss: 177.53712463378906
[step: 450] loss: 0.02152777463197708
[step: 451] loss: 215.48193359375
[step: 451] loss: 0.0215277299284935
[step: 452] loss: 313.54803466796875
[step: 452] loss: 0.021527716889977455
[step: 453] loss: 423.2911376953125
[step: 453] loss: 0.02152768

...

[step: 1000] loss: 119.40397644042969
[step: 1000] loss: 0.02152092009782791
[step: 1001] loss: 119.41532897949219
[step: 1001] loss: 0.021520905196666718
[step: 1002] loss: 119.10696411132812
[step: 1002] loss: 0.021520892158150673
[step: 1003] loss: 118.9024887084961
[step: 1003] loss: 0.02152087725698948
[step: 1004] loss: 118.70751190185547
[step: 1004] loss: 0.021520864218473434
[step: 1005] loss: 118.53813171386719
[step: 1005] loss: 0.02152084931731224
[step: 1006] loss: 118.38594055175781
[step: 1006] loss: 0.021520836278796196
[step: 1007] loss: 118.31998443603516
[step: 1007] loss: 0.02152082696557045
[step: 1008] loss: 118.15583801269531
[step: 1008] loss: 0.021520812064409256
[step: 1009] loss: 118.00633239746094
[step: 1009] loss: 0.02152080088853836
[step: 1010] loss: 117.87873840332031
[step: 1010] loss: 0.021520784124732018
[step: 1011] loss: 117.71987915039062
[step: 1011] loss: 0.021520771086215973
[step: 1012] loss: 117.57400512695312
[step: 1012] loss: 0.02152075991

[step: 1107] loss: 0.021519070491194725
[step: 1108] loss: 128.45118713378906
[step: 1108] loss: 0.021519048139452934
[step: 1109] loss: 128.4195556640625
[step: 1109] loss: 0.021519025787711143
[step: 1110] loss: 128.40548706054688
[step: 1110] loss: 0.021519001573324203
[step: 1111] loss: 128.40396118164062
[step: 1111] loss: 0.021518977358937263
[step: 1112] loss: 128.41290283203125
[step: 1112] loss: 0.021518955007195473
[step: 1113] loss: 128.417236328125
[step: 1113] loss: 0.021518932655453682
[step: 1114] loss: 128.4364776611328
[step: 1114] loss: 0.021518904715776443
[step: 1115] loss: 128.4776153564453
[step: 1115] loss: 0.021518882364034653
[step: 1116] loss: 128.55584716796875
[step: 1116] loss: 0.021518860012292862
[step: 1117] loss: 128.73095703125
[step: 1117] loss: 0.021518832072615623
[step: 1118] loss: 129.04904174804688
[step: 1118] loss: 0.021518807858228683
[step: 1119] loss: 129.70033264160156
[step: 1119] loss: 0.021518783643841743
[step: 1120] loss: 130.858688354

[step: 1215] loss: 0.02151462249457836
[step: 1216] loss: 559.5452880859375
[step: 1216] loss: 0.021514538675546646
[step: 1217] loss: 600.6736450195312
[step: 1217] loss: 0.02151445485651493
[step: 1218] loss: 360.251220703125
[step: 1218] loss: 0.021514372900128365
[step: 1219] loss: 144.47021484375
[step: 1219] loss: 0.021514281630516052
[step: 1220] loss: 336.3755187988281
[step: 1220] loss: 0.02151418849825859
[step: 1221] loss: 326.1029052734375
[step: 1221] loss: 0.02151408977806568
[step: 1222] loss: 163.9744415283203
[step: 1222] loss: 0.021513987332582474
[step: 1223] loss: 307.0421142578125
[step: 1223] loss: 0.02151387929916382
[step: 1224] loss: 196.49490356445312
[step: 1224] loss: 0.021513773128390312
[step: 1225] loss: 212.5830078125
[step: 1225] loss: 0.021513663232326508
[step: 1226] loss: 197.56561279296875
[step: 1226] loss: 0.021513544023036957
[step: 1227] loss: 198.4558868408203
[step: 1227] loss: 0.021513422951102257
[step: 1228] loss: 181.76303100585938
[step: 

[step: 1322] loss: 0.02149353176355362
[step: 1323] loss: 129.1678009033203
[step: 1323] loss: 0.02149282954633236
[step: 1324] loss: 128.07058715820312
[step: 1324] loss: 0.02149263024330139
[step: 1325] loss: 127.3113784790039
[step: 1325] loss: 0.021491581574082375
[step: 1326] loss: 127.061767578125
[step: 1326] loss: 0.021490223705768585
[step: 1327] loss: 127.26451110839844
[step: 1327] loss: 0.021489331498742104
[step: 1328] loss: 127.73368835449219
[step: 1328] loss: 0.021488619968295097
[step: 1329] loss: 128.2710418701172
[step: 1329] loss: 0.021487627178430557
[step: 1330] loss: 128.8170928955078
[step: 1330] loss: 0.021486470475792885
[step: 1331] loss: 129.2587890625
[step: 1331] loss: 0.021485313773155212
[step: 1332] loss: 129.75604248046875
[step: 1332] loss: 0.021484194323420525
[step: 1333] loss: 130.2361602783203
[step: 1333] loss: 0.021483080461621284
[step: 1334] loss: 131.03790283203125
[step: 1334] loss: 0.021481895819306374
[step: 1335] loss: 132.03575134277344


[step: 1431] loss: 128.74575805664062
[step: 1431] loss: 0.018322177231311798
[step: 1432] loss: 130.71255493164062
[step: 1432] loss: 0.014147138223052025
[step: 1433] loss: 130.97677612304688
[step: 1433] loss: 0.01630307361483574
[step: 1434] loss: 129.0385284423828
[step: 1434] loss: 0.027924364432692528
[step: 1435] loss: 127.69547271728516
[step: 1435] loss: 0.024781646206974983
[step: 1436] loss: 128.45672607421875
[step: 1436] loss: 0.017868632450699806
[step: 1437] loss: 129.45042419433594
[step: 1437] loss: 0.021660655736923218
[step: 1438] loss: 129.4049072265625
[step: 1438] loss: 0.026263507083058357
[step: 1439] loss: 128.29388427734375
[step: 1439] loss: 0.02230914682149887
[step: 1440] loss: 127.26849365234375
[step: 1440] loss: 0.018662771210074425
[step: 1441] loss: 127.23728942871094
[step: 1441] loss: 0.020424436777830124
[step: 1442] loss: 127.84590148925781
[step: 1442] loss: 0.021315347403287888
[step: 1443] loss: 128.38400268554688
[step: 1443] loss: 0.018624361